<a href="https://colab.research.google.com/github/willewiik/Text-Mining/blob/main/pythonscript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/willewiik/text-mining.git
%cd "text-mining"

Cloning into 'text-mining'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 22 (delta 3), reused 15 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 850.16 KiB | 6.21 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/text-mining


In [11]:
# FROM CHATPGT, how to convert R lists.RDS to Python dict
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

# Activate pandas conversion for data frames if needed
pandas2ri.activate()

# Load the RDS file
readRDS = ro.r['readRDS']
list_report = readRDS("combined_stats_report_list.RDS")

# Convert R list to Python list
list_report_py = list(list_report)

# If elements are R lists, convert them to Python dicts
list_report_py = [
    dict(zip(item.names, list(item))) if hasattr(item, "names") else list(item)
    for item in list_report_py
]

In [12]:
print(type(list_report_py))
print(len(list_report_py))
print(type(list_report_py[0]))
if isinstance(list_report_py[0], dict):
    print(list_report_py[0].keys())  # List the keys of the first dictionary

for key, value in list_report_py[0].items():
    print(f"{key}: {value}")

<class 'list'>
158
<class 'dict'>
dict_keys(['hteam', 'ateam', 'starttime', 'score', 'home_players', 'away_players', 'report'])
hteam: [1] "Manchester United"

ateam: [1] "Fulham"

starttime: [1] "2024-08-16T19:00:00.000Z"

score: [1 0]
home_players:          ID                Name Position FotMob.rating Minutes.played Goals  \
1    611491         Andre Onana  Starter           8.7             90     0   
2    775539   Noussair Mazraoui  Starter           7.8             81     0   
3    255610       Harry Maguire  Starter          7.66             81     0   
4    847983   Lisandro Martinez  Starter          7.63             90     0   
5    751550         Diogo Dalot  Starter          7.68             90     0   
6    208494            Casemiro  Starter          8.01             90     0   
7   1292810       Kobbie Mainoo  Starter          7.79             84     0   
8   1070052         Amad Diallo  Starter          7.03             61     0   
9    750032         Mason Mount  Start

In [10]:
from transformers import pipeline

# Load sentiment-analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model = "distilbert-base-uncased-finetuned-sst-2-english")

# Example usage
example_text = "Ismaila Sarr scored twice as Crystal Palace secured a vital victory."
result = sentiment_analyzer(example_text)
print(result)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9996942281723022}]


In [31]:
import re

# Load a pre-trained sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model = "distilbert-base-uncased-finetuned-sst-2-english")

# Example: Processing a single report from `list_report_py`
def extract_sentiment_for_players(report, home_players, away_players):
    # Step 1: Tokenize the report into sentences
    sentences = re.split(r'(?<=[.!?]) +', report)

    # Step 2: Initialize player_sentiments dictionary, including only players
    # who played more than 9 minutes (needs minimum of 10 minutes to get a fotmob score)
    player_sentiments = {
        player['Name']: []
        for player in home_players + away_players
        if int(player.get('Minutes.played', 0)) > 9
    }

    # Step 3: Analyze each sentence
    for sentence in sentences:
        # Perform sentiment analysis
        sentiment = sentiment_pipeline(sentence)[0]

        # Check which players are mentioned in the sentence
        for player in player_sentiments:
            if player in sentence:
                # Add the sentiment score to the player's list
                player_sentiments[player].append(sentiment['label'])

    # Step 4: Aggregate sentiment scores per player
    aggregated_sentiments = {
        player: {
            "Positive": sentiments.count("POSITIVE"),
            "Negative": sentiments.count("NEGATIVE"),
        }
        for player, sentiments in player_sentiments.items()
    }

    return aggregated_sentiments



Device set to use cpu


In [32]:
# TEST of the first report, Manchester United vs Fulham, 16 aug, 21:00
first_report = list_report_py[0]

# Convert the R StrVector to a native Python string
report_text = str(first_report['report'][0])  # If it's a single-element StrVector

# Convert DataFrames to lists of dictionaries
home_players = first_report['home_players'].to_dict('records')
away_players = first_report['away_players'].to_dict('records')


sentiments = extract_sentiment_for_players(
    report_text,
    home_players,
    away_players
)

In [33]:
sentiments

{'Andre Onana': {'Positive': 1, 'Negative': 0},
 'Noussair Mazraoui': {'Positive': 1, 'Negative': 0},
 'Harry Maguire': {'Positive': 0, 'Negative': 1},
 'Lisandro Martinez': {'Positive': 0, 'Negative': 0},
 'Diogo Dalot': {'Positive': 0, 'Negative': 0},
 'Casemiro': {'Positive': 0, 'Negative': 1},
 'Kobbie Mainoo': {'Positive': 0, 'Negative': 0},
 'Amad Diallo': {'Positive': 1, 'Negative': 0},
 'Mason Mount': {'Positive': 0, 'Negative': 1},
 'Marcus Rashford': {'Positive': 1, 'Negative': 0},
 'Bruno Fernandes': {'Positive': 1, 'Negative': 0},
 'Alejandro Garnacho': {'Positive': 1, 'Negative': 0},
 'Joshua Zirkzee': {'Positive': 1, 'Negative': 0},
 'Bernd Leno': {'Positive': 1, 'Negative': 0},
 'Kenny Tete': {'Positive': 1, 'Negative': 0},
 'Issa Diop': {'Positive': 0, 'Negative': 0},
 'Calvin Bassey': {'Positive': 1, 'Negative': 0},
 'Antonee Robinson': {'Positive': 1, 'Negative': 0},
 'Andreas Pereira': {'Positive': 0, 'Negative': 1},
 'Sasa Lukic': {'Positive': 0, 'Negative': 0},
 'A